In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as nplin
import itertools
import timeit

In [2]:
np.random.seed(0)

In [3]:
def operators(s):
    #generate terms in the energy function
    n_seq,n_var = s.shape
    ops = np.zeros((n_seq,n_var+int(n_var*(n_var-1)/2.0)))

    jindex = 0
    for index in range(n_var):
        ops[:,jindex] = s[:,index]
        jindex +=1

    for index in range(n_var-1):
        for index1 in range(index+1,n_var):
            ops[:,jindex] = s[:,index]*s[:,index1]
            jindex +=1
            
    return ops

In [4]:
def energy_ops(ops,w):
    return np.sum(ops*w[np.newaxis,:],axis=1)

In [5]:
def generate_seqs(n_var,n_seq,n_sample=30,g=1.0):
    samples = np.random.choice([1.0,-1.0],size=(n_seq*n_sample,n_var),replace=True)
    ops = operators(samples)
    n_ops = ops.shape[1]

    #w_true = g*(np.random.rand(ops.shape[1])-0.5)/np.sqrt(float(n_var))
    w_true = np.random.normal(0.,g/np.sqrt(n_var),size=n_ops)

    sample_energy = energy_ops(ops,w_true)
    p = np.exp(sample_energy)
    p /= np.sum(p)
    out_samples = np.random.choice(np.arange(n_seq*n_sample),size=n_seq,replace=True,p=p)
    
    return w_true,samples[out_samples] #,p[out_samples],sample_energy[out_samples]

In [6]:
def eps_machine(s,eps_scale=0.1,max_iter=151,alpha=0.1):
    MSE = np.zeros(max_iter)
    KL = np.zeros(max_iter)
    E_av = np.zeros(max_iter)
    
    n_seq,n_var = s.shape
    ops = operators(s)
    n_ops = ops.shape[1]
    cov_inv = np.eye(ops.shape[1])

    np.random.seed(13)
    w = np.random.rand(n_ops)-0.5    
    
    #w_iter = np.zeros((max_iter,n_ops))
    for i in range(max_iter): 
        
        eps_scale = np.random.rand()/np.max([1.,np.max(np.abs(w))])
        
        energies_w = energy_ops(ops,w)
        probs_w = np.exp(-energies_w*(1-eps_scale))
        z_data = np.sum(probs_w)
        probs_w /= z_data
        ops_expect_w = np.sum(probs_w[:,np.newaxis]*ops,axis=0)
        
        #if i%int(10) == 0:
        E_exp = (probs_w*energies_w).sum()
        #KL[i] = -E_exp - np.log(z_data) + np.sum(np.log(np.cosh(w*eps_scale))) + n_var*np.log(2.)
        
        #if i > 50 and KL[i] >= KL[i-1]: break
                
        E_av[i] = energies_w.mean()
        MSE[i] = ((w-w_true)**2).mean()
        sec_order = w*eps_scale
        w += alpha*cov_inv.dot((ops_expect_w - sec_order))        
        
        #w_iter[i,:] = w
        
    #return MSE,KL,E_av,w_iter
    return MSE,-E_av,w

In [7]:
def hopfield_model(s):
    ops = operators(s)
    w = np.mean(ops,axis=0)
    #print('hopfield error ',nplin.norm(w-w_true))
    return w

In [8]:
def boltzmann_machine_exact(s,s_all,max_iter=150,alpha=5e-2,cov=False):
    n_seq,n_var = s.shape
    ops = operators(s)
    cov_inv = np.eye(ops.shape[1])
    ops_obs = np.mean(ops,axis=0)
    ops_model = operators(s_all)

    n_ops = ops.shape[1]
    
    np.random.seed(13)
    w = np.random.rand(n_ops)-0.5    
    for iterate in range(max_iter):
        energies_w = energy_ops(ops_model,w)
        probs_w = np.exp(energies_w)
        probs_w /= np.sum(probs_w)
        if iterate%10 == 0: 
            #print(iterate,nplin.norm(w-w_true)) #,nplin.norm(spin_cov_w-spin_cov_obs))
            MSE = ((w-w_true)**2).mean()
            #print(iterate,MSE)
            
        w += alpha*cov_inv.dot(ops_obs - np.sum(ops_model*probs_w[:,np.newaxis],axis=0))

    #print('final',iterate,MSE)

    return w

In [9]:
#print('Hopfield:')
#w1 = hopfield_model(seqs)
#print(((w1-w_true)**2).mean())

In [10]:
nn_var = 16
n_var_list = np.linspace(10,40,nn_var).astype(int)
n_var_list

array([10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40])

In [11]:
max_iter = 100

n_seq = 10000
#n_var = 10
g = 1.

run_time1 = np.zeros(nn_var)
run_time2 = np.zeros(nn_var)
for i,n_var in enumerate(n_var_list):
    n_var = int(n_var)
    w_true,seqs = generate_seqs(n_var,n_seq,g=g)

    print('esp machine:')
    start_time1 = timeit.default_timer()
    MSE,E_av,w = eps_machine(seqs,eps_scale=0.1,max_iter=max_iter)
    run_time1[i] = timeit.default_timer() - start_time1
    print('run_time:',n_var,run_time1[i],MSE[-1])

    #print('Boltzmann (exact):')
    #start_time2 = timeit.default_timer()
    #s_all = np.asarray(list(itertools.product([1.0, -1.0], repeat=n_var)))
    #w2 = boltzmann_machine_exact(seqs,s_all,cov=False)
    #run_time2[i] = timeit.default_timer() - start_time2
    #print('run_time:',run_time2[i])

esp machine:
run_time: 10 0.2887086699993233 0.004250046974839709
esp machine:
run_time: 12 0.278752987000189 0.005423912438294265
esp machine:
run_time: 14 0.3632008169988694 0.0068510145653338786
esp machine:
run_time: 16 0.5024194689976866 0.004409782646752482
esp machine:
run_time: 18 0.6710007760011649 0.004162376780515645
esp machine:
run_time: 20 0.9437748489981459 0.004485328009463353
esp machine:
run_time: 22 1.2373842200031504 0.007682508603072752
esp machine:
run_time: 24 1.5601243729979615 0.0047537264880975075
esp machine:
run_time: 26 1.9236514900003385 0.006082236647057643
esp machine:
run_time: 28 2.2409028059992124 0.006555197872134502
esp machine:
run_time: 30 3.4441917779986397 0.01053632715455088
esp machine:
run_time: 32 3.9611615169997094 0.006064910835230919
esp machine:
run_time: 34 4.526621379001881 0.01268488082037958
esp machine:
run_time: 36 5.111497290999978 0.005888088428956886
esp machine:
run_time: 38 5.948709934000362 0.007930488855160091
esp machine:
r

In [12]:
print(run_time1)

[0.28870867 0.27875299 0.36320082 0.50241947 0.67100078 0.94377485
 1.23738422 1.56012437 1.92365149 2.24090281 3.44419178 3.96116152
 4.52662138 5.11149729 5.94870993 6.33454964]


In [13]:
np.savetxt('run_time_eps_random.txt',(n_var_list,run_time1),fmt='%f')